# Trains models with specified parameter settings, saved trained models

In [ ]:
retina_width_options = [1, 32]
vvs_layers_options = [0, 2, 4]
num_trials = 10
data_set = 'pixel'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']
test_conditions = ['Same', 'Diff', 'NoPix']
fresh_data = 0
data_augmentation = 0
pretrain = 1
n_gpus = 1

In [ ]:
import os
from tqdm import tqdm_notebook

if pretrain:
    pretrain_data_set = 'cifar10'
    trial_label = 'PRETRAIN'
    print("Pretraining model!")
    for vvs_layers in tqdm_notebook(vvs_layers_options, desc="VVS Layers", leave=True):
        for retina_out_width in tqdm_notebook(retina_width_options, desc="Retina Out Width", leave=False):
            pretrain_model_name = f"{pretrain_data_set}_{trial_label}_vvs_layers={vvs_layers}_retina_out_channels={retina_out_width}"
            args = (f"--data_set {data_set} --trial_label {trial_label} "
                f"--vvs_layers {vvs_layers} --retina_out_width {retina_out_width} "
        #         f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                f"--model_name {pretrain_model_name}")
            os.system(f'python3 TrainModel.py {args}')

In [ ]:
import os
from tqdm import tqdm_notebook

start_trial = 1

for trial in tqdm_notebook(range(start_trial, 1+num_trials), desc='Trial'):
    for noise_type in tqdm_notebook(noise_types, desc='Noise type', leave=True):
        trial_label = f"{noise_type}_{trial}"
        for vvs_layers in tqdm_notebook(vvs_layers_options, desc="VVS Layers", leave=True):
            for retina_out_width in tqdm_notebook(retina_width_options, desc="Retina Out Width", leave=True):
                model_name = f"{data_set}_{trial_label}_vvs_layers={vvs_layers}_retina_out_channels={retina_out_width}"
                args = (f"--data_set {data_set} --trial_label {trial_label} "
                        f"--vvs_layers {vvs_layers} --retina_out_width {retina_out_width} "
                        f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                        f"--n_gpus {n_gpus} --model_name {model_name}")
                if pretrain:
                    pretrain_data_set = 'cifar10'
                    pretrain_model_name = f"{pretrain_data_set}_PRETRAIN_vvs_layers={vvs_layers}_retina_out_channels={retina_out_width}"
                    args += f" --pretrained_model {pretrain_model_name}"
                os.system(f'python3 TrainModel.py {args}')

In [ ]:
# Load accuracy scores and plot

%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Defaults
# filter_size = 9
# retina_layers = 2
# vvs_width = 32
n_epochs = 20

rows = []
test_rows = []

for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
        trial_label = f"{noise_type}_{trial}"
        for vvs_layers in vvs_layers_options:
            for retina_out_width in retina_width_options:

                model_name = f"{data_set}_{trial_label}_vvs_layers={vvs_layers}_retina_out_channels={retina_out_width}"

                acc_scores = np.load(os.path.join('Logs', f'{model_name}_ACC.npy'))
                valacc_scores = np.load(os.path.join('Logs', f'{model_name}_VALACC.npy'))
                loss = np.load(os.path.join('Logs', f'{model_name}_LOSS.npy'))
                valloss = np.load(os.path.join('Logs', f'{model_name}_VALLOSS.npy'))

                with open(os.path.join('Logs', f'{model_name}_CONDVALACC.json'), "r") as jf:
                    cond_acc = json.load(jf)
                with open(os.path.join('Logs', f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                    cond_loss = json.load(jf)

                for condition in test_conditions:
                    test_rows.append({'Trial': trial, 'Noise Type': noise_type,
                                     'VVS Layers': vvs_layers, 'Retina Out Width': retina_out_width,
                                     'Condition': condition, 'Loss': cond_loss[condition], 
                                      'Accuracy': cond_acc[condition]})
                for epoch in range(n_epochs):
                    rows.append({'Trial': trial, 'Noise Type': noise_type, #'Noise Condition': noise_cond, 
                                 'VVS Layers': vvs_layers, 'Retina Out Width': retina_out_width,
                                 'Evaluation': 'Testing', 'Epoch': epoch+1, 'Loss': valloss[epoch], 
                                 'Accuracy': valacc_scores[epoch]})

                    rows.append({'Trial': trial, 'Noise Type': noise_type, #'Noise Condition': noise_cond, 
                                 'VVS Layers': vvs_layers, 'Retina Out Width': retina_out_width,
                                 'Evaluation': 'Training', 'Epoch': epoch+1, 'Loss': loss[epoch], 
                                 'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise Type', 'VVS Layers', 'Retina Out Width', 
                                     'Evaluation', 'Epoch', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise Type', 'VVS Layers', 'Retina Out Width', 
                                             'Condition', 'Loss', 'Accuracy'])
# scores
# test_scores

In [ ]:
# fig = plt.figure()
g = sns.relplot(x='Epoch', y='Accuracy', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)
# g = sns.relplot(x='Epoch', y='Loss', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)

In [ ]:
fig = plt.figure(figsize=(18,6))
g = sns.catplot(x="Condition", y="Accuracy", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)
# g = sns.catplot(x="Condition", y="Loss", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)

In [ ]:
metrics = ['Loss', 'Accuracy']
fig, axes = plt.subplots(nrows=len(metrics), ncols=len(noise_types), sharex=True, sharey='row', figsize=(16,8))
for t, noise_type in enumerate(noise_types):
    for m, metric in enumerate(metrics):
        sns.lineplot(x='Epoch', y=metric, hue='Evaluation', style='Retina Out Width', data=scores[(scores['Noise Type'] == noise_type)], ax=axes[m, t])#.query(f"'Noise Type' == '{noise_type}' and 'Noise Condition' == '{noise_cond}'"))
        if m == 0:
            axes[m, t].set_title(noise_type)
        axes[m, t].set_ylabel(metric)

In [ ]:
metrics = ['Loss', 'Accuracy']
fig, axes = plt.subplots(nrows=len(vvs_layers_options), ncols=len(noise_types), sharex=True, sharey='row', figsize=(16,12))
for t, noise_type in enumerate(noise_types):
    for v, layers in enumerate(vvs_layers_options):
        sns.lineplot(x='Epoch', y='Accuracy', hue='Evaluation', style='Retina Out Width', 
                     data=scores[(scores['Noise Type'] == noise_type) & (scores['VVS Layers'] == layers)], ax=axes[v, t])
        if v == 0:
            axes[v, t].set_title(noise_type)
        axes[v, t].set_ylabel(f'VVS Layers = {layers}')

# Plot Receptive Fields

In [ ]:
import os

import numpy as np
from tqdm import tqdm_notebook

plot_layers = ['retina_2', 'vvs_1']
start_trial = 1
end_trial = 10

# vvs_layers = 4
# layer = 'retina_2'

for trial in tqdm_notebook(range(start_trial, 1+end_trial), desc='Trial'):
    for noise_type in tqdm_notebook(noise_types, desc='Noise type', leave=True):
        trial_label = f"{noise_type}_{trial}"
        for vvs_layers in tqdm_notebook(vvs_layers_options, desc="VVS Layers", leave=True):

            for retina_out_width in tqdm_notebook(retina_width_options, desc="Retina Out Width", leave=True):
                model_name = f"{data_set}_{trial_label}_vvs_layers={vvs_layers}_retina_out_channels={retina_out_width}"
                for layer in plot_layers:
                    if layer == 'vvs_1' and vvs_layers == 0:
                        continue
                    args = (f"--data_set {data_set} --trial_label {trial_label} "
                            f"--vvs_layers {vvs_layers} --retina_out_width {retina_out_width} "
                            f"--layer_name {layer} --model_name {model_name}")
                    os.system(f'python3 VisualizeRFs.py {args}')

# Plot Performance on original stimuli

In [ ]:
retina_width_options = [1, 32]  # [1, 2, 4, 8, 16, 32]
vvs_layers_options = [0, 1, 2, 3, 4]
num_trials = 1  # 10
data_set = 'cifar10'

In [ ]:
import os
from tqdm import tqdm_notebook

for trial in tqdm_notebook(range(1, num_trials+1), desc="Trial"):
    trial_label = 'Trial'+str(trial)
    for vvs_layers in tqdm_notebook(vvs_layers_options, desc="VVS Layers"):
        for retina_out_width in tqdm_notebook(retina_width_options, desc="Retina Out Width", leave=False):
#             print('Trial ', trial, 'VVS Layers ', vvs_layers, 'Retina Out Width ', retina_out_width)
            args = (f"--trial_label {trial_label} --vvs_layers {vvs_layers} --retina_out_width {retina_out_width}")
            os.system(f"python3 TrainModel.py {args}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plot_line_format_list = []
plot_line_format_list.append(('k', '-'))
plot_line_format_list.append(([.5, .5, .5], '--'))

dict = {}
dict_err = {}

cur_format = -1

for retina_out_width in retina_width_options:
    cur_format += 1
    dict[retina_out_width] = []
    dict_err[retina_out_width] = []

    for vvs_layers in vvs_layers_options:
        trials = []
        for trial in range(1, num_trials+1):
            trial_label = 'Trial'+str(trial)

            model_name = (
                f"{data_set}_type_{trial_label}_filter_size_{filter_size}"
                f"_retina_layers_{retina_layers}_vvs_layers{vvs_layers}"
                f"_retina_out_channels_{retina_out_width}_vvs_width_{vvs_width}"
                f"_epochs_{n_epochs}"
            )

#             acc = np.load(os.path.join('Logs', f'SAVED_{model_name}_ACC.npy'))[-1]
            val_acc = np.load(os.path.join('Logs', f'SAVED_{model_name}_VALACC.npy'))[-1]
#             val_acc = np.load('Logs/SAVED_cifar10_type_'+trial_label+'_noise_start_'+str(0.0)+'_noise_end_'+str(0.0)+'_reg_'+str(0.0)+'_retina_reg_'+str(0.0)+'_retina_hidden_channels_'+str(32)+'_SS_'+str(1)+'_task_'+'classification'+'_filter_size_'+str(9)+'_retina_layers_'+str(2)+'_vvs_layers'+str(vvs_layers)+'_bias_'+str(1)+'_actreg_'+str(0.0)+'_retina_out_channels_'+str(retina_out_width)+'_vvs_width_'+str(32)+'_epochs_'+str(20)+'_VALACC.npy')[-1]
            trials.append(val_acc)

        dict[retina_out_width].append(np.mean(trials))
        dict_err[retina_out_width].append(np.std(trials))
        print(trials)

plt.figure(figsize=(4, 8))
for retina_out_width in retina_width_options:
    if retina_out_width == 1:
        plt.errorbar(range(len(vvs_layers_options)), dict[retina_out_width], dict_err[retina_out_width], c='k', marker='o')
    elif retina_out_width == 32:
        plt.errorbar(range(len(vvs_layers_options)), dict[retina_out_width], dict_err[retina_out_width], c='gray', marker='o')
    else:
        plt.errorbar(range(len(vvs_layers_options)), dict[retina_out_width], dict_err[retina_out_width], marker='o')


plt.legend([str(w)+' retina output channels' for w in retina_width_options])
plt.xlabel('VVS-net Layers')
plt.ylabel('Cifar-10 Test Set Performance')
plt.xticks(range(len(vvs_layers_options)), vvs_layers_options)

plt.show()